In [2]:
## 규원 05.09 중앙일보 클롱링 csv파일 저장 
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
from tqdm import tqdm  # tqdm 추가
import time

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 5, 9)


## 날짜 형식을 바꿔주는 함수 
def date_range(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

## 중앙일보 url을 넘겨주면, 해당 url에 있는 기사 list를 가져와주는 함수 
def fetch_news_articles(url):
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles_url = []
    for article in soup.find_all('li', class_='card'):
        title_link = article.find('a')
        if title_link and title_link.has_attr('href'):
            articles_url.append(title_link['href'])
            
    return articles_url

## 요청받은 url에서 url, 제목, 기사내용으로 변형해서 return 해주는 함수  
def fetch_articles_with_details(url):
    # URL에서 웹 페이지를 요청하고 응답을 받습니다.
    response = requests.get(url)
    response.raise_for_status()  # HTTP 에러가 있을 경우 예외를 발생시킵니다.
    
    # 응답으로부터 HTML을 파싱합니다.
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 기사 데이터를 저장할 리스트를 초기화합니다.
    articles = []
    
    # 각 기사를 찾아내어 필요한 정보를 추출합니다.
    for article in soup.find_all('article'):
        # 제목 추출
        headline = article.find('h1', class_='headline')
        title = headline.get_text(strip=True) if headline else "No title"
        # 본문 내용 추출
        body = article.find('div', class_='article_body fs3')
        text = []
        if body:
            for p in body.find_all('p', attrs={"data-divno": True}):
                text.append(p.get_text(strip=True))
        text_content = ' '.join(text)
        
        # 기사 URL, 제목, 본문을 튜플로 만들어 리스트에 추가합니다.
        print(url+"진행중")
        articles.append([url, title, text_content])
    
    return articles


# 모든 날짜에 대해 URL을 생성하고 크롤링을 실행, 모든 뉴스 링크 list 전달 
def get_news_links():
    ## 시작 요일부터 하루씩 올려가며, 데이터 크롤링 
    links_url = []
    for single_date in date_range(start_date, end_date):
        formatted_date = single_date.strftime("%Y/%m/%d")
        url = f'https://www.joongang.co.kr/sitemap/index/{formatted_date}'
        links = fetch_news_articles(url)
        print(f"Date: {formatted_date}")
        for link in links:
            links_url.append(link)
    ## 모든 크롤링한 모든 링크의 url 전달 
    return links_url


## 최종적으로 url 제목 본분 행태로 저장해주는 코드 
def joongang_crawler():
    crawled_data = []
    
    ## 모아진 뉴스 링크 가져옴 
    links_list = get_news_links()
    for link in links_list:
        ## 링크에 방문해서 뉴스기사 url 제목 뉴스기사 추출해서 저장
        links_detail = fetch_articles_with_details(link)
        crawled_data.extend(links_detail) # list에 추가,append로 하면 3차원 배열됨  
    #print(crawled_data)
    return crawled_data


def save_to_csv(data):
    # 결과를 저장할 폴더 생성
    result_dir = 'result'
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    # CSV 파일 경로 설정
    csv_file_path = os.path.join(result_dir, 'joongang_newsTest.csv')
    
    # CSV 파일을 열고 데이터를 저장합니다.
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # CSV 파일의 헤더를 작성합니다.
        writer.writerow(['URL', 'Title', 'Content'])
        # 데이터를 CSV 파일에 작성합니다.
        writer.writerows(data)
        
# 데이터가 예시로 준비되어 있다고 가정하고, 이 함수를 사용하여 저장합니다.
example_data = [
    ('https://example.com/article1', 'Title 1', 'Content of article 1...'),
    ('https://example.com/article2', 'Title 2', 'Content of article 2...'),
    ('https://example.com/article3', 'Title 3', 'Content of article 3...'),
    ('https://example.com/gitACtionTest', 'Title 3', 'Content of article 3...'),
    ('https://example.com/article3', 'Title 3', 'Content of article 3...')
]

# CSV 파일 저장
if __name__ == "__main__":
    start_time = time.time()  # 코드 실행 시작 시간 기록
    crawled_data = joongang_crawler()
    save_to_csv(example_data)
    end_time = time.time()  # 코드 실행 종료 시간 기록
    print(f"Execution time: {end_time - start_time} seconds")  # 실행 시간 출력

Execution time: 0.0020859241485595703 seconds
